In [ ]:
#NOTE
#i run this code in csc virtual enfironment
#so it may require some preparartion steps (i.e. with files upload) that are described below

In [1]:
! pip install mxnet==1.5.1

     |████████████████████████████████| 23.1 MB 2.7 MB/s eta 0:00:01/s eta 0:00:08█████████▋                      | 7.0 MB 2.7 MB/s eta 0:00:07�██████████▊                | 11.3 MB 2.7 MB/s eta 0:00:0515.8 MB 2.7 MB/s eta 0:00:03��███▍    | 19.8 MB 2.7 MB/s eta 0:00:02
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.13.2
    Uninstalling graphviz-0.13.2:
      Successfully uninstalled graphviz-0.13.2


In [2]:
! pip install opencv-python

     |████████████████████████████████| 60.3 MB 187 kB/s eta 0:00:01    |██▍                             | 4.5 MB 4.2 MB/s eta 0:00:14                     | 9.4 MB 4.2 MB/s eta 0:00:13 MB 4.2 MB/s eta 0:00:11��████▍                     | 19.6 MB 4.2 MB/s eta 0:00:10          | 27.8 MB 4.2 MB/s eta 0:00:0832.2 MB 4.2 MB/s eta 0:00:07MB/s eta 0:00:06��        | 45.3 MB 4.2 MB/s eta 0:00:04MB/s eta 0:00:03��█████████████████████████▎  | 55.1 MB 4.2 MB/s eta 0:00:02�██▋| 59.6 MB 4.2 MB/s eta 0:00:01


In [3]:
! pip install requests matplotlib opencv-python-headless

     |████████████████████████████████| 47.6 MB 2.1 MB/s eta 0:00:01              | 1.4 MB 2.1 MB/s eta 0:00:2221███▋                          | 8.3 MB 2.1 MB/s eta 0:00:19█████████                       | 13.3 MB 2.1 MB/s eta 0:00:17██▌                   | 18.7 MB 2.1 MB/s eta 0:00:14�██████████▋                | 23.2 MB 2.1 MB/s eta 0:00:12MB/s eta 0:00:09ta 0:00:07███████████████████████▊      | 38.3 MB 2.1 MB/s eta 0:00:05███████▏  | 43.4 MB 2.1 MB/s eta 0:00:02


In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx

In [2]:
import warnings
warnings.filterwarnings("ignore", category=Warning) 

#### Task 1 (5 points):

go to https://github.com/miaow1988/SqueezeNet_v1.2 and download the 'symbol.json' and '.params' files (there is not a 'synset.txt' file! so don't use these lines, Hint: just comment these lines).

* Install MXNet v1.5 (hint: create a new conda environmet with python 3, pip install mxnet==1.5.1) and follow the same steps of the lecture (part: *Using pre-trained models as feature extractors*). Find the flatten output layer and create a feature extractor (hint: It should be a numpy array of 1000 elements).
* Download the dogs versus cats *training folder* from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data (Remember the number of images is 12500 for each class).
* Extract the array of features for different number of images (N: 10, 100, 500, 1000, also 5000 and 12500) and for each value train your favorite binary classifier (only one!!!) using GridSearch to optimize some hyperparameters. Consider to use https://notebooks.csc.fi if you have computational limitations. 

* Report the accuracy for each value of N and the computational time during the training step.

In [25]:
path='https://github.com/miaow1988/SqueezeNet_v1.2/'
[mx.test_utils.download(path+'raw/master/model-symbol.json'),
 mx.test_utils.download(path+'raw/master/model-0000.params')]

['model-symbol.json', 'model-0000.params']

In [26]:
sym, arg_params, aux_params = mx.model.load_checkpoint('model', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

In [27]:
# list the last 10 layers
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['fire9_concat_output',
 'dropout0_output',
 'conv10_conv_weight',
 'conv10_conv_bias',
 'conv10_conv_output',
 'conv10_relu_output',
 'pool10_output',
 'flatten0_output',
 'softmax_label',
 'softmax_output']

In [28]:
fe_sym = all_layers['flatten0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [29]:
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

In [8]:
from time import time
from subprocess import run

In [15]:
! pip install slugify

  Created wheel for slugify: filename=slugify-0.0.1-py3-none-any.whl size=1908 sha256=04bc3e7b2df5357107557a4d9ca778be04639fc286a5e8d9df95e2c493db4eb9
  Stored in directory: /home/jovyan/.cache/pip/wheels/d4/7b/0d/bd65011a1b442843bb4043e396f727ab0f1e76050355b9156a
Successfully built slugify


In [30]:
#upload kaggle.json file beforehand
start = time()
script = f'''
    pip install --upgrade --force-reinstall --no-deps kaggle;
    mkdir -p ~/.kaggle;
    cp ~/kaggle.json ~/.kaggle/;
    chmod 600 ~/.kaggle/kaggle.json;
    kaggle competitions download --quiet -c dogs-vs-cats-redux-kernels-edition;
    unzip -o dogs-vs-cats-redux-kernels-edition.zip;
    unzip -o train.zip;
    '''
proc = run(script,
          shell=True,
          text=True,
          capture_output=True,
          timeout=120)
print(proc.stderr)
print(f'runtime {time()-start:.0f} sec')


runtime 15 sec


In [31]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

In [32]:
# from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data
from os import listdir
from os.path import isfile, join
import os

mypath = join(os.getcwd(),'train')

cats_imgs = [join(mypath,f) for f in listdir(mypath) if f.startswith('cat')]
dogs_imgs = [join(mypath,f) for f in listdir(mypath) if f.startswith('dog')]

In [33]:
import cv2
def get_image(url, show=False):
    if url.startswith('http'):
        # download and show the image
        fname = mx.test_utils.download(url)
    else:
        fname = url
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

In [34]:
#with N = 10 , number of features

Nmax = 10
cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax]]
dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax]]

Y_cats = np.array(Nmax * [1])
Y_dogs = np.array(Nmax * [0])

X_cvd = np.vstack([cats_features,dogs_features])
Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()

In [22]:
#use favourite classifier model
#LogisticRegression?


In [35]:
#finding best parameters for classifier

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'tol': [0.0001,0.001,0.01,0.1,1],
              'C': [0.1,1,10,50],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'multi_class': ['auto', 'ovr', 'multinomial']}

grid_search = GridSearchCV(LogisticRegression(max_iter=3000), param_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)
grid_search.fit(X_train, y_train)

print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}\n".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Test set score: 1.00
Best parameters: {'C': 1, 'multi_class': 'multinomial', 'solver': 'sag', 'tol': 0.1}
Best cross-validation score: 1.00

Best estimator:
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.1, verbose=0,
                   warm_start=False)


In [36]:
#calculate time and accuracy for different number of features, using best parameters

Nmax = [10, 100, 500, 1000, 5000, 12500]
for i in Nmax:
    start = time()
    
    cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:i]]
    dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:i]]

    Y_cats = np.array(i * [1])
    Y_dogs = np.array(i * [0])

    X_cvd = np.vstack([cats_features,dogs_features])
    Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()
    
    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)
    
    lg = LogisticRegression(**grid_search.best_params_, max_iter=3000).fit(X_train, y_train)
    print(str(i)+' features:')
    print(f'runtime {time() - start:.0f} seconds')
    print('test set score: {:.2f}'.format(lg.score(X_test, y_test)))
    print('-'*20)

10 features:
runtime 0 seconds
test set score: 1.00
--------------------
100 features:
runtime 4 seconds
test set score: 0.94
--------------------
500 features:
runtime 17 seconds
test set score: 0.90
--------------------
1000 features:
runtime 35 seconds
test set score: 0.95
--------------------
5000 features:
runtime 171 seconds
test set score: 0.96
--------------------
12500 features:
runtime 431 seconds
test set score: 0.96
--------------------


#### Task 2 (5 points):

Repeat all previous steps using MobileNet V2 (https://github.com/KeyKy/mobilenet-mxnet). How the two networks compare in terms of accuracy and running time?

**WARNING**: At least for N= 5000 and 12500 it can take some time in your computer, depending of your resources. The time can largely increases depending of your chosen classifier.

In [32]:
path='https://github.com/KeyKy/mobilenet-mxnet/'
[mx.test_utils.download(path+'raw/master/mobilenet_v2-symbol.json'),
 mx.test_utils.download(path+'raw/master/mobilenet_v2-0000.params')]

['mobilenet_v2-symbol.json', 'mobilenet_v2-0000.params']

In [33]:
with open('mobilenet_v2-symbol.json', 'r') as f:
    for n in range (10):
        s = f.readline().strip()
        if s: print(s)

{
"nodes": [
{
"op": "null",
"name": "data",
"inputs": []
},
{
"op": "null",
"name": "conv1_weight",


In [ ]:
#change manually the output layer to 'fc7_flatten_output' in .json file

In [37]:
sym, arg_params, aux_params = mx.model.load_checkpoint('mobilenet_v2', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

In [38]:
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['conv6_4_bn_moving_var',
 'conv6_4_bn_output',
 'relu6_4_output',
 'pool6_output',
 'fc7_weight',
 'fc7_bias',
 'fc7_output',
 'fc7_flatten_output',
 'prob_label',
 'prob_output']

In [39]:
fe_sym = all_layers['fc7_flatten_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [40]:
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

In [41]:
Nmax = 10
cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax]]
dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax]]

Y_cats = np.array(Nmax * [1])
Y_dogs = np.array(Nmax * [0])

X_cvd = np.vstack([cats_features,dogs_features])
Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()

In [42]:
param_grid = {'tol': [0.0001,0.001,0.01,0.1,1],
              'C': [0.1,1,10,50],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'multi_class': ['auto', 'ovr', 'multinomial']}

grid_search = GridSearchCV(LogisticRegression(max_iter=3000), param_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)
grid_search.fit(X_train, y_train)

print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}\n".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Test set score: 0.20
Best parameters: {'C': 0.1, 'multi_class': 'auto', 'solver': 'liblinear', 'tol': 0.01}
Best cross-validation score: 0.67

Best estimator:
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.01, verbose=0,
                   warm_start=False)


In [43]:
#calculate time and accuracy for different number of features and best params

Nmax = [10, 100, 500, 1000, 5000, 12500]
for i in Nmax:
    start = time()
    
    cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:i]]
    dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:i]]

    Y_cats = np.array(i * [1])
    Y_dogs = np.array(i * [0])

    X_cvd = np.vstack([cats_features,dogs_features])
    Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()
    
    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)
    
    lg = LogisticRegression(**grid_search.best_params_, max_iter=3000).fit(X_train, y_train)
    print(str(i)+' features:')
    print(f'runtime {time() - start:.0f} seconds')
    print('test set score: {:.2f}'.format(lg.score(X_test, y_test)))
    print('-'*20)

10 features:
runtime 1 seconds
test set score: 0.20
--------------------
100 features:
runtime 13 seconds
test set score: 0.48
--------------------
500 features:
runtime 68 seconds
test set score: 0.62
--------------------
1000 features:
runtime 128 seconds
test set score: 0.67
--------------------
5000 features:
runtime 657 seconds
test set score: 0.68
--------------------
12500 features:
runtime 1746 seconds
test set score: 0.69
--------------------
